# Clustering

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import sklearn
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

## Charger les données

Le but, ici, est juste de charger les données dans Python. Pour traiter les données "tabulaires", on utilise toujours `pandas` qui est une librairie extrêment puissante pour la manipulation de données tabulaires.

Le "path" que je mets ici est celui qui correspond à ma machine. Les données sont mises dans un dossier qui s'appelle `data`, puis dans le sous-dossier `clustering`.

Cette fois-ci, par contre, nous n'avons qu'un seul jeu de données car celui-ci sera utilisé pour entraîner le modèle et pour segmenter les clients.

In [5]:
data_path = '../data/clustering'

df = pd.read_csv(os.path.join(data_path, 'customers.csv'))

In [6]:
df.head()

,ID,Gender,Ever_Married,Age,Graduated,Profession,Work_Experience,Spending_Score,Family_Size,Var_1
0,462809,Male,No,22,No,Healthcare,1.0,Low,4.0,Cat_4
1,462643,Female,Yes,38,Yes,Engineer,NaN,Average,3.0,Cat_4
2,466315,Female,Yes,67,Yes,Engineer,1.0,Low,1.0,Cat_6
3,461735,Male,Yes,67,Yes,Lawyer,0.0,High,2.0,Cat_6
4,462669,Female,Yes,40,Yes,Entertainment,NaN,High,6.0,Cat_6


La colonne `ID` n'est utilisée que pour identifier les différents clients. Cependant, il est possible d'utiliser toutes les autres valeurs. On voit, par contre, qu'on a déjà des données manquantes dans la colonne `Work_Experience`. 

Du coup, il nous faut faire une visualisation des données pour essayer de gagner un peu de connaissance. 

## Visualisation des données

In [8]:
# Visualisation des colonnes numériques

num_cols = ['Age', 'Work_Experience', 'Family_Size']

for col in num_cols:
    fig = px.histogram(df, x=col, title=f'Distribution de {col}', histnorm='percent')

    fig.update_layout(
        xaxis_title_text=col,
        yaxis_title_text='Pourcentage',
    )

    fig.show()

In [9]:
df.head()

,ID,Gender,Ever_Married,Age,Graduated,Profession,Work_Experience,Spending_Score,Family_Size,Var_1
0,462809,Male,No,22,No,Healthcare,1.0,Low,4.0,Cat_4
1,462643,Female,Yes,38,Yes,Engineer,NaN,Average,3.0,Cat_4
2,466315,Female,Yes,67,Yes,Engineer,1.0,Low,1.0,Cat_6
3,461735,Male,Yes,67,Yes,Lawyer,0.0,High,2.0,Cat_6
4,462669,Female,Yes,40,Yes,Entertainment,NaN,High,6.0,Cat_6


In [10]:
# Visualisation des données catégoriques

cat_cols = ['Gender', 'Ever_Married', 'Graduated', 'Profession', 'Spending_Score', 'Var_1']

for col in cat_cols:

    fig = px.histogram(df, x=col, title=f'Distribution de {col}', histnorm='percent')

    fig.update_layout(
        xaxis_title_text=col,
        yaxis_title_text='Pourcentage',
    )

    fig.show()

Étant donné que nous n'avons pas de labels pour savoir quels groupes de clients on a, il est intéressant de voir les relations entre les différentes variables. 

Vous pouvez changer les variables ci-dessous pour voir les relations qui peuvent les lier.

In [12]:
feature1 = 'Age'
feature2 = 'Var_1'

fig = px.scatter(df, x=feature1, y=feature2, title=f'{feature1} vs {feature2}')

fig.update_layout(
    xaxis_title_text=feature1,
    yaxis_title_text=feature2,
)

fig.show()

Difficile de trouver un cluster visuellement... 

Du coup, essayons tout de même d'en trouver en utilisant un algorithme simple. Cependant, il va nous falloir transformer les données pour que l'on puisse utiliser ces algorithmes.

## Entraînement du modèle

On utilise la même pipeline de préparation des données que pour la classification.

In [13]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder

In [14]:
df.head()

,ID,Gender,Ever_Married,Age,Graduated,Profession,Work_Experience,Spending_Score,Family_Size,Var_1
0,462809,Male,No,22,No,Healthcare,1.0,Low,4.0,Cat_4
1,462643,Female,Yes,38,Yes,Engineer,NaN,Average,3.0,Cat_4
2,466315,Female,Yes,67,Yes,Engineer,1.0,Low,1.0,Cat_6
3,461735,Male,Yes,67,Yes,Lawyer,0.0,High,2.0,Cat_6
4,462669,Female,Yes,40,Yes,Entertainment,NaN,High,6.0,Cat_6


In [15]:
num_features = ['Age', 'Work_Experience', 'Family_Size']
cat_features = ['Gender', 'Ever_Married', 'Graduated', 'Profession', 'Spending_Score', 'Var_1']

features = df[num_features + cat_features]

In [16]:
# Numerical transformer
numerical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler())
])

# Categorical transformer
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

In [17]:
# Le ColumnTransformer est une pipeline qui me permet d'appliquer les transformations sur tous les datasets que je peux avoir à utiliser
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, num_features),
        ('cat', categorical_transformer, cat_features)
    ])

In [ ]:
features_preprocessed = preprocessor.fit_transform(features)
